# **London Underground ETL**

## Objectives

* Clean and merge Footfall and Station Coordinates datasets

## Inputs

* To run this notebook the StationFootfall and Stations datasets are required 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [111]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\jackr\\OneDrive'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [112]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [113]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\jackr'

# Extract

* Retrieving datasets and checking values

Importing necessary libraries

In [114]:
import pandas as pd

Parsing CSV files into DataFrames

In [115]:
Footfall = pd.read_csv(r'c:\Users\jackr\OneDrive\Desktop\my_projects\Project-1\Dataset\Dirty\StationFootfall_2024_2025 .csv')
Station_Coords = pd.read_csv(r'c:\Users\jackr\OneDrive\Desktop\my_projects\Project-1\Dataset\Dirty\Stations_20180921.csv')

Checking variable types 

In [116]:
Footfall.info()
Station_Coords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249121 entries, 0 to 249120
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   TravelDate     249121 non-null  int64 
 1   DayOfWeek      249121 non-null  object
 2   Station        249121 non-null  object
 3   EntryTapCount  249121 non-null  int64 
 4   ExitTapCount   249121 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 9.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FID       479 non-null    int64  
 1   OBJECTID  479 non-null    int64  
 2   NAME      479 non-null    object 
 3   EASTING   479 non-null    int64  
 4   NORTHING  479 non-null    int64  
 5   LINES     271 non-null    object 
 6   NETWORK   479 non-null    object 
 7   Zone      479 non-null    int64  
 8   x         479 non-null    float6

^ The 'x' and 'y' values are floats, this is ideal for us as it's necessary in creating map scatter plots later on

Displaying head of each DataFrame

In [117]:
Footfall.head()

,TravelDate,DayOfWeek,Station,EntryTapCount,ExitTapCount
0,20240101,Monday,Abbey Road DLR,395,375
1,20240101,Monday,Abbey Wood,5898,5963
2,20240101,Monday,Acton Central,609,474
3,20240101,Monday,Acton Main Line,1717,1710
4,20240101,Monday,Acton Town,2928,3334


In [118]:
Station_Coords.head()

,FID,OBJECTID,NAME,EASTING,NORTHING,LINES,NETWORK,Zone,x,y
0,0,78,Temple,530959,180803,"District, Circle",London Underground,1,-0.112644,51.510474
1,1,79,Blackfriars,531694,180893,"District, Circle",London Underground,1,-0.102020,51.511114
2,2,80,Mansion House,532354,180932,"District, Circle",London Underground,1,-0.092495,51.511306
3,3,81,Cannon Street,532611,180900,"District, Circle",London Underground,1,-0.088801,51.510963
4,4,82,Monument,532912,180824,"District, Circle",London Underground,1,-0.084502,51.510209


---

# Section 2

* Merging and Cleaning

Filtering non-underground stations

In [119]:
Station_Coords = Station_Coords.query("NETWORK == 'London Underground'")

# Checking Networks have been filtered correctly          
Station_Coords.query("NETWORK == ['London Overground', 'Tramlink', 'DLR', 'TfL Rail']")
                            

,FID,OBJECTID,NAME,EASTING,NORTHING,LINES,NETWORK,Zone,x,y


^ No rows appear when querying our filtered stations, they are no longer in the DataFrame

Checking for duplicate values

In [120]:
Footfall.duplicated().sum()
Station_Coords.duplicated().sum()

0

Cleaning whitespace in columns and values

In [121]:
Footfall.columns = Footfall.columns.str.strip()
Station_Coords.columns = Station_Coords.columns.str.strip()
Footfall['Station'] = Footfall['Station'].str.strip()
Station_Coords['NAME'] = Station_Coords['NAME'].str.strip()


Merging Underground datasets

In [122]:
merged_ug = pd.merge(
    Footfall,
    Station_Coords[['NAME', 'LINES', 'NETWORK', 'Zone', 'x', 'y']],
    left_on='Station',
    right_on='NAME',
    how='left'
)

merged_ug.head(10)

,TravelDate,DayOfWeek,Station,EntryTapCount,ExitTapCount,NAME,LINES,NETWORK,Zone,x,y
0,20240101,Monday,Abbey Road DLR,395,375,NaN,NaN,NaN,NaN,NaN,NaN
1,20240101,Monday,Abbey Wood,5898,5963,NaN,NaN,NaN,NaN,NaN,NaN
2,20240101,Monday,Acton Central,609,474,NaN,NaN,NaN,NaN,NaN,NaN
3,20240101,Monday,Acton Main Line,1717,1710,NaN,NaN,NaN,NaN,NaN,NaN
4,20240101,Monday,Acton Town,2928,3334,Acton Town,"District, Piccadilly",London Underground,3.0,-0.278433,51.502137
5,20240101,Monday,Aldgate,7223,7382,Aldgate,"Metropolitan, Circle",London Underground,1.0,-0.074236,51.513982
6,20240101,Monday,Aldgate East,10657,11723,Aldgate East,"Hammersmith & City, District",London Underground,1.0,-0.069540,51.514917
7,20240101,Monday,All Saints,641,578,NaN,NaN,NaN,NaN,NaN,NaN
8,20240101,Monday,Alperton,2117,2235,Alperton,Piccadilly,London Underground,4.0,-0.298361,51.540227
9,20240101,Monday,Amersham,793,815,Amersham,Metropolitan,London Underground,9.0,-0.606147,51.673662


Dropping duplicate name column

In [123]:
merged_ug = merged_ug.drop(columns=['NAME'], errors='ignore')
merged_ug.head()

,TravelDate,DayOfWeek,Station,EntryTapCount,ExitTapCount,LINES,NETWORK,Zone,x,y
0,20240101,Monday,Abbey Road DLR,395,375,NaN,NaN,NaN,NaN,NaN
1,20240101,Monday,Abbey Wood,5898,5963,NaN,NaN,NaN,NaN,NaN
2,20240101,Monday,Acton Central,609,474,NaN,NaN,NaN,NaN,NaN
3,20240101,Monday,Acton Main Line,1717,1710,NaN,NaN,NaN,NaN,NaN
4,20240101,Monday,Acton Town,2928,3334,"District, Piccadilly",London Underground,3.0,-0.278433,51.502137


Removing all rows with NaN values, these are non-underground stations

In [124]:
merged_ug = merged_ug.dropna()
merged_ug.head()

,TravelDate,DayOfWeek,Station,EntryTapCount,ExitTapCount,LINES,NETWORK,Zone,x,y
4,20240101,Monday,Acton Town,2928,3334,"District, Piccadilly",London Underground,3.0,-0.278433,51.502137
5,20240101,Monday,Aldgate,7223,7382,"Metropolitan, Circle",London Underground,1.0,-0.074236,51.513982
6,20240101,Monday,Aldgate East,10657,11723,"Hammersmith & City, District",London Underground,1.0,-0.069540,51.514917
8,20240101,Monday,Alperton,2117,2235,Piccadilly,London Underground,4.0,-0.298361,51.540227
9,20240101,Monday,Amersham,793,815,Metropolitan,London Underground,9.0,-0.606147,51.673662


Establishing new index

In [125]:
merged_ug = merged_ug.set_index('Station')
merged_ug.head()

,TravelDate,DayOfWeek,EntryTapCount,ExitTapCount,LINES,NETWORK,Zone,x,y
Station,,,,,,,,,
Acton Town,20240101,Monday,2928,3334,"District, Piccadilly",London Underground,3.0,-0.278433,51.502137
Aldgate,20240101,Monday,7223,7382,"Metropolitan, Circle",London Underground,1.0,-0.074236,51.513982
Aldgate East,20240101,Monday,10657,11723,"Hammersmith & City, District",London Underground,1.0,-0.069540,51.514917
Alperton,20240101,Monday,2117,2235,Piccadilly,London Underground,4.0,-0.298361,51.540227
Amersham,20240101,Monday,793,815,Metropolitan,London Underground,9.0,-0.606147,51.673662


Changing Zone values to categorical variables

In [130]:
merged_ug['Zone'] = merged_ug['Zone'].astype(int).astype('category')
merged_ug.tail(400)

,TravelDate,DayOfWeek,EntryTapCount,ExitTapCount,LINES,NETWORK,Zone,x,y
Station,,,,,,,,,
High Barnet,20250801,Friday,4637,4516,Northern,London Underground,5,-0.192561,51.650029
High Street Kensington,20250801,Friday,14341,13575,"District, Circle",London Underground,1,-0.190607,51.499935
Highbury & Islington,20250801,Friday,20044,20331,Victoria,London Underground,2,-0.102126,51.546918
Highgate,20250801,Friday,5974,5738,Northern,London Underground,3,-0.144861,51.577342
Hillingdon,20250801,Friday,1997,1833,"Metropolitan, Piccadilly",London Underground,6,-0.447812,51.553449
...,...,...,...,...,...,...,...,...,...
Wimbledon Park,20250802,Saturday,2175,2111,District,London Underground,3,-0.197491,51.433537
Wood Green,20250802,Saturday,10056,9514,Piccadilly,London Underground,3,-0.108247,51.596971
Wood Lane,20250802,Saturday,6930,7297,"Hammersmith & City, Circle",London Underground,2,-0.221994,51.509632


Changing TravelDate to correct datetime format

In [ ]:
merged_ug['TravelDate'] = pd.to_datetime(merged_ug['TravelDate'], format='%Y%m%d', errors='coerce')
merged_ug.head()

,TravelDate,DayOfWeek,EntryTapCount,ExitTapCount,LINES,NETWORK,Zone,x,y
Station,,,,,,,,,
Acton Town,2024-01-01,Monday,2928,3334,"District, Piccadilly",London Underground,3,-0.278433,51.502137
Aldgate,2024-01-01,Monday,7223,7382,"Metropolitan, Circle",London Underground,1,-0.074236,51.513982
Aldgate East,2024-01-01,Monday,10657,11723,"Hammersmith & City, District",London Underground,1,-0.069540,51.514917
Alperton,2024-01-01,Monday,2117,2235,Piccadilly,London Underground,4,-0.298361,51.540227
Amersham,2024-01-01,Monday,793,815,Metropolitan,London Underground,9,-0.606147,51.673662


Checking if column types are as expected

In [127]:
merged_ug.dtypes

TravelDate          int64
DayOfWeek          object
EntryTapCount       int64
ExitTapCount        int64
LINES              object
NETWORK            object
Zone             category
x                 float64
y                 float64
dtype: object

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [128]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)


IndentationError: expected an indented block after 'try' statement on line 2 (553063055.py, line 5)